In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

In [ ]:
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/SemEval2016-Task6-raw-annotations-stance.csv')

In [ ]:
import socket, pickle, numpy as np
import struct
import math

In [ ]:
import gensim.downloader as api

wv = api.load('glove-wiki-gigaword-300')

[==================================================] 100.0% 376.1/376.1MB downloaded


In [ ]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['Stance'] = encoder.fit_transform(df['Stance'])

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.utils import resample
import re

In [ ]:

def rnn_wordvec(df, wv):
  
    # Separate the majority and minority classes
    df_majority = df[df['Stance'] == 0]
    df_minority = df[df['Stance'] == 1]

    # Resample the majority class to match the number of samples in the minority class
    df_majority_resampled = resample(df_majority, replace=False, n_samples=len(df_minority), random_state=42)

    # Concatenate the resampled majority class with the minority class
    df_resampled = pd.concat([df_majority_resampled, df_minority])

    # Shuffle the rows of the resampled dataset
    df_resampled = df_resampled.sample(frac=1, random_state=42)
    df = df_resampled

    def clean_data(review):
        review = re.sub('[^a-zA-Z]', ' ', review)
        review = review.lower()
        return review

    df['Tweet'] = df['Tweet'].apply(clean_data)

    def remove_stop_words(review):
        review_minus_sw = []
        stop_words = stopwords.words('english')
        review = review.split()
        review = [review_minus_sw.append(word) for word in review if word not in stop_words]
        review = ' '.join(review_minus_sw)
        return review

    df['Tweet'] = df['Tweet'].apply(remove_stop_words)

    def lematize(review):
        lemmatizer = WordNetLemmatizer()
        review = review.split()
        review = [lemmatizer.lemmatize(word) for word in review]
        review = ' '.join(review)
        return review

    df['Tweet'] = df['Tweet'].apply(lematize)

    message = df['Tweet']
    stance = df['Stance']

    words = []
    for sent in message:
        sent_token = sent_tokenize(sent)
        for sent in sent_token:
            words.append(simple_preprocess(sent)) 

    # model = Word2Vec(words, size=100, window=5, min_count=1, workers=4)
    word_indeces = [[wv.vocab[word].index for word in sentence if word in wv.vocab] for sentence in words]
    input_data = pad_sequences(word_indeces, maxlen=100, padding='post')
    X_train, X_test , y_train, y_test = train_test_split(input_data, stance, test_size=0.20)

    max_length = 100
    vocab_size = len(wv.vocab)

    def define_rnn_model():
       model = Sequential()
       model.add(Embedding(vocab_size, 100, input_length=max_length))
       model.add(LSTM(128))
       model.add(Dense(1, activation='sigmoid'))
       model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
       return model

    # call the define_rnn_model function to create the model object
    model = define_rnn_model()

    # print the model summary
    model.summary()
    #calling adam optimizer
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    
    #visualizing each eppoch
    history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
  
    #caluclating loss and accuracy
    loss, accuracy = model.evaluate(X_test,y_test)

    #printing accuracy
    print('Testing Accuracy is {} '.format(accuracy*100))
    
    y_prob = model.predict(X_test)
    y_predict = (y_prob>=0.5).astype(int)
    from sklearn.metrics import classification_report
    print(classification_report(y_test,y_predict,target_names = ["Against","Favor"]))
rnn_wordvec(df,wv)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 100, 100)          40000000  
                                                                 
 lstm_9 (LSTM)               (None, 128)               117248    
                                                                 
 dense_9 (Dense)             (None, 1)                 129       
                                                                 
Total params: 40,117,377
Trainable params: 40,117,377
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
578/578 [==============================] - 626s 1s/step - loss: 0.6933 - acc: 0.5046 - val_loss: 0.6934 - val_acc: 0.4927
Epoch 2/10
578/578 [==============================] - 629s 1s/step - loss: 0.6933 - acc: 0.5002 - val_loss: 0.6932 - val_acc: 0.4927
Epoch 3/10
578/578 [====================

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
for i in df["Target"].unique():
  dfa = df[df["Target"]==i]
  print("........",i,".........")
  rnn_wordvec(dfa,wv)

........ Legalization of Abortion .........
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 100)          40000000  
                                                                 
 lstm_4 (LSTM)               (None, 128)               117248    
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 40,117,377
Trainable params: 40,117,377
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
108/108 [==============================] - 120s 1s/step - loss: 0.6943 - acc: 0.4735 - val_loss: 0.6931 - val_acc: 0.5058
Epoch 2/10
108/108 [==============================] - 118s 1s/step - loss: 0.6939 - acc: 0.4828 - val_loss: 0.6931 - val_acc: 0.5

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 100, 100)          40000000  
                                                                 
 lstm_5 (LSTM)               (None, 128)               117248    
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 40,117,377
Trainable params: 40,117,377
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
114/114 [==============================] - 124s 1s/step - loss: 0.6939 - acc: 0.5008 - val_loss: 0.6931 - val_acc: 0.5055
Epoch 2/10
114/114 [==============================] - 123s 1s/step - loss: 0.6934 - acc: 0.4992 - val_loss: 0.6931 - val_acc: 0.5055
Epoch 3/10
114/114 [====================

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 100, 100)          40000000  
                                                                 
 lstm_6 (LSTM)               (None, 128)               117248    
                                                                 
 dense_6 (Dense)             (None, 1)                 129       
                                                                 
Total params: 40,117,377
Trainable params: 40,117,377
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
177/177 [==============================] - 191s 1s/step - loss: 0.6937 - acc: 0.4917 - val_loss: 0.6932 - val_acc: 0.4979
Epoch 2/10
177/177 [==============================] - 189s 1s/step - loss: 0.6934 - acc: 0.4988 - val_loss: 0.6932 - val_acc: 0.5021
Epoch 3/10
177/177 [====================

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

     Against       0.00      0.00      0.00       705
       Favor       0.50      1.00      0.67       711

    accuracy                           0.50      1416
   macro avg       0.25      0.50      0.33      1416
weighted avg       0.25      0.50      0.34      1416

........ Atheism .........
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 100, 100)          40000000  
                                                                 
 lstm_7 (LSTM)               (None, 128)               117248    
                                                                 
 dense_7 (Dense)             (None, 1)                 129       
                                                                 
Total params: 40,117,377
Trainable params: 40,117,377
Non-trainable params: 0
__________________

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

     Against       0.47      1.00      0.64       274
       Favor       0.00      0.00      0.00       313

    accuracy                           0.47       587
   macro avg       0.23      0.50      0.32       587
weighted avg       0.22      0.47      0.30       587

........ Donald Trump .........
Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 100, 100)          40000000  
                                                                 
 lstm_8 (LSTM)               (None, 128)               117248    
                                                                 
 dense_8 (Dense)             (None, 1)                 129       
                                                                 
Total params: 40,117,377
Trainable params: 40,117,377
Non-trainable params: 0
_____________

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
